In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
# 添加时间维度
def add_timestamp(data):
    data = data.copy()
    data['report_date'] = pd.to_datetime(data['report_date'], format= "%Y%m%d")
    data['day'] = data['report_date'].dt.day
    data['month'] = data['report_date'].dt.month
    data['year'] = data['report_date'].dt.year
    data['week'] = data['report_date'].dt.week
    data['weekday'] = data['report_date'].dt.weekday
    return data

In [3]:
# 求purhcase和redeem按天的总量
def get_total_balance(data, date):
    df_tmp = data.copy()
    df_tmp = df_tmp.groupby(['report_date'])['total_purchase_amt','total_redeem_amt'].sum()
    df_tmp.reset_index(inplace=True)
    return df_tmp[(df_tmp['report_date']>= date)].reset_index(drop=True)

In [4]:
# 生成测试数据
def generate_test_data(data):
    total_balance = data.copy()
    start = datetime.datetime(2014,9,1)
    end = datetime.datetime(2014,10,1)
    testdata = []
    while start != end:
        temp = [start, np.nan, np.nan]
        testdata.append(temp)
        start += datetime.timedelta(days = 1)
    testdata = pd.DataFrame(testdata)
    testdata.columns = total_balance.columns

    total_balance = pd.concat([total_balance, testdata], axis = 0)
    total_balance = total_balance.reset_index(drop=True)
    return total_balance.reset_index(drop=True)

# 读取数据和数据准备

In [5]:
data = pd.read_csv('./Purchase Redemption Data/user_balance_table.csv')

In [6]:
# 添加时间属性
data = add_timestamp(data)

<ipython-input-2-8d29299f16eb>:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  data['week'] = data['report_date'].dt.week


In [7]:
# 得到按天的总量统计
total_balance = get_total_balance(data, date='2014-03-01')

<ipython-input-3-823597d2f371>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_tmp = df_tmp.groupby(['report_date'])['total_purchase_amt','total_redeem_amt'].sum()


In [8]:
# 续上测试数据
total_balance = generate_test_data(total_balance)

In [9]:
# 把新生成的数据也进行时间拆分
total_balance = add_timestamp(total_balance)

<ipython-input-2-8d29299f16eb>:8: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  data['week'] = data['report_date'].dt.week


In [10]:
tmp_data = total_balance.copy()

# 构造时间因子
- 计算周期因子

In [13]:
# (test)计算每周一到周日的purchase和redeem均值
total_balance = tmp_data[tmp_data['report_date'] < datetime.datetime(2014,9,1)].copy()
weekday_weight = total_balance[['weekday', 'total_purchase_amt', 'total_redeem_amt']].groupby('weekday', as_index=False).mean()

In [15]:
weekday_weight.columns=['weekday', 'purchase_weekday', 'redeem_weekday']

In [16]:
# 用周日期均值除以总均值得到周期因子
weekday_weight['purchase_weekday'] /= np.mean(total_balance['total_purchase_amt'])
weekday_weight['redeem_weekday'] /= np.mean(total_balance['total_redeem_amt'])

In [17]:
total_balance = pd.merge(total_balance, weekday_weight, on='weekday', how='left')

- 计算日期因子

In [18]:
# 统计周一到周日在1-31号出现的频次weekday，day出现的频次
weekday_count = total_balance[['report_date','day','weekday']].groupby(['day','weekday'],as_index=False).count()
weekday_count = pd.merge(weekday_count, weekday_weight, on='weekday')

In [19]:
# 日期计算 = 周期因子 * （周一到周日在1-31号出现的频次） / 一共有几个月份
weekday_count['purchase_weekday'] = weekday_count['purchase_weekday']* weekday_count['report_date'] / len(np.unique(total_balance['month']))
weekday_count['redeem_weekday'] = weekday_count['redeem_weekday']* weekday_count['report_date'] / len(np.unique(total_balance['month']))

In [20]:
# 得到日期因子
day_rate = weekday_count.drop(['weekday','report_date'], axis=1).groupby('day', as_index=False).sum()

In [21]:
# 计算日期平均值
day_mean = total_balance[['day','total_purchase_amt','total_redeem_amt']].groupby('day', as_index=False).mean()

In [22]:
day_base = pd.merge(day_mean, day_rate, on='day', how='left')
# 去掉日期因子影响之后的base值
day_base['total_purchase_amt'] /= day_base['purchase_weekday']
day_base['total_redeem_amt'] /= day_base['redeem_weekday']

In [23]:
for index, row in day_base.iterrows():
        if row['day'] == 31:
            break
        day_base.loc[index, 'report_date'] = pd.to_datetime('2014/9/'+ str(int(row['day'])))

In [24]:
# 利用周期因子：base * 周期因子 = 预测结果
day_base['weekday'] = day_base['report_date'].dt.weekday

In [25]:
day_pred = day_base[['day','total_purchase_amt','total_redeem_amt','report_date','weekday']]
day_pred = pd.merge(day_pred, weekday_weight, on='weekday')

In [26]:
day_pred['total_purchase_amt'] *= day_pred['purchase_weekday']
day_pred['total_redeem_amt'] *= day_pred['redeem_weekday']

In [27]:
# 按照日期排序
day_pred = day_pred.sort_values('report_date')[['report_date','total_purchase_amt','total_redeem_amt']]

# 准备提交数据

In [29]:
# 整理日期格式
day_pred['report_date'] = day_pred['report_date'].apply(lambda x: str(x).replace('-','')[0:8])

In [30]:
day_pred.to_csv('Submission/rule_base_1.csv', index=False, header=None)